In [1]:
!pip install pymysql

In [2]:
!pip install pykrx

    high ,#고가
    low ,#저가
    end_close #종가
    trading_volume #거래량
    transaction_amount  #거래대금
    end_rate_change #등락율
    institutional_total #기관 합계
    other_corporations #기타 법인
    individual #개인
    foreigner_total # 외국인 합계
    short_selling ,#공매도
    short_buying # 공매도 매수
    short_importance #공매도 비중

In [3]:
import joblib
import pymysql
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt 
import seaborn as sns
import platform
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier

from pykrx import stock

In [4]:
conn = pymysql.connect(host='localhost', user='root', password='a026195', db='stockai', charset='utf8', cursorclass=pymysql.cursors.DictCursor)
curs = conn.cursor()
stock_code_sql = "SELECT DISTINCT stock_code FROM StockData"
curs.execute(stock_code_sql)
stock_code = curs.fetchall()
stock_code_list = pd.DataFrame(stock_code)

# 총합 정확도 예측을 위한 변수
total = 0

#출력하고자 하는 주식 개수 입력, 최대 100개
stock_n = 100

try:
    # for문에 
    for i in range(0,stock_n):

        curs = conn.cursor()
        sql = "SELECT * FROM StockData WHERE stock_code ='{}'".format(str(stock_code_list['stock_code'][i]))
        # 코드 앞에서 A 지우고 주식 이름 불러오기
        name = stock.get_market_ticker_name(str(stock_code_list['stock_code'][i][1:]))
        curs.execute(sql)
        data = curs.fetchall()
        
        df = pd.DataFrame(data)
        
        # 0.00등 Nan 값으로 인식되지 않는 결측치를 Nan으로 바꿈
        df["start_open"] = df["start_open"].apply(lambda x: np.nan if x<0.00001 else x)
        df["high"] = df["high"].apply(lambda x: np.nan if x<0.00001 else x)
        df["low"] = df["low"].apply(lambda x: np.nan if x<0.00001 else x)
        df["trading_volume"] = df["trading_volume"].apply(lambda x: np.nan if x<0.00001 else x)
        df["institutional_total"] = df["institutional_total"].apply(lambda x: np.nan if x<0.00001 else x)
        df["other_corporations"] = df["other_corporations"].apply(lambda x: np.nan if x<0.00001 else x)
        df["individual"] = df["individual"].apply(lambda x: np.nan if x<0.00001 else x)
        df["foreigner_total"] = df["foreigner_total"].apply(lambda x: np.nan if x<0.00001 else x)
        df["transaction_amount"] = df["transaction_amount"].apply(lambda x: np.nan if x<0.00001 else x)
        df["short_selling"] = df["short_selling"].apply(lambda x: np.nan if x<0.01 else x)
        df["short_buying"] = df["short_buying"].apply(lambda x: np.nan if x<0.01 else x)
          
        df = df.fillna(method='bfill')
        df = df.fillna(method='ffill')
    
        df['1yd_end_close'] = df['end_close'].shift(1)
        df['1yd_trading'] = df['trading_volume'].shift(1)
        df['5일 평균 종가'] = df['end_close'].rolling(5).mean()
        df['10일 평균 종가'] = df['end_close'].rolling(10).mean()
        df['20일 평균 종가'] = df['end_close'].rolling(20).mean()
        df['60일 평균 종가'] = df['end_close'].rolling(60).mean()
        df['120일 평균 종가'] = df['end_close'].rolling(120).mean()
    
        df['5일 평균 거래량'] = df['trading_volume'].rolling(5).mean()
        df['10일 평균 거래량'] = df['trading_volume'].rolling(10).mean()
        df['20일 평균 거래량'] = df['trading_volume'].rolling(20).mean()
        df['60일 평균 거래량'] = df['trading_volume'].rolling(60).mean()
        df['120일 평균 거래량'] = df['trading_volume'].rolling(120).mean()
    
        pd.set_option('mode.chained_assignment',  None)
    
        # X1 일 뒤 X2일 평균 종가
        X1 = 10
        X2 = 5
        df['X1일 뒤 종가'] = df['end_close'].shift(-X1)
        df['X1일 뒤 X2일 평균 종가'] = 0
        for i in range(0, df.shape[0]-X2):
            for j in range(0,X2):
                df['X1일 뒤 X2일 평균 종가'][i] = df['X1일 뒤 X2일 평균 종가'][i] + df['X1일 뒤 종가'][i+j]
            
        df['X1일 뒤 X2일 평균 종가'] = df['X1일 뒤 X2일 평균 종가']/X2
    
        df['X1일 뒤 X2일 평균 종가 변화량'] = df['X1일 뒤 X2일 평균 종가'] - df['end_close']
    
        # 분류 모델 쓸 때 0,1로 라벨링 하기
        df['X1일 뒤 X2일 평균 종가 변화량'] = df['X1일 뒤 X2일 평균 종가 변화량'].apply(lambda x: 1 if x>0 else 0)

        df.dropna(inplace=True)
        sql2 = "SELECT * FROM MacroeconomicIndicators"
        curs.execute(sql2)
        data2 = curs.fetchall()

        df2 = pd.DataFrame(data2)

        df2['코스피 5일 평균 종가'] = df2['kospi'].rolling(5).mean()
        df2['코스피 20일 평균 종가'] = df2['kospi'].rolling(20).mean()
        df2['코스피 60일 평균 종가'] = df2['kospi'].rolling(60).mean()
        df2['코스피 120일 평균 종가'] = df2['kospi'].rolling(120).mean()
    
        df2.dropna(inplace=True)
        
        df_m = pd.merge(df, df2, left_on='data_time',right_on='date_time',how='inner')
        
        #merge하면서 생기는 id 행들 제거
        df_m.drop(columns=['id_x','id_y'],inplace=True)
                                            
        # 컬럼 제작
        df_m['전일종가대비 당일 시가비율'] = df_m['1yd_end_close']/df_m['start_open']
        df_m['당일종가 대비 당일 고가 비율'] = df_m['high']/df_m['end_close']
        df_m['당일종가 대비 당일 저가 비율'] = df_m['low']/df_m['end_close']
        df_m['당일 종가 대비 전일 종가 비율'] = df_m['end_close']/df_m['1yd_end_close']
        df_m['전일 거래량 대비 당일 거래량 비율'] = df_m['trading_volume']/df_m['1yd_trading']
        df_m['5일 평균 종가 대비 당일 종가 비율'] = df_m['end_close']/df_m['5일 평균 종가']
        df_m['10일 평균 종가 대비 당일 종가 비율'] = df_m['end_close']/df_m['10일 평균 종가']
        df_m['20일 평균 종가 대비 당일 종가 비율'] = df_m['end_close']/df_m['20일 평균 종가']
        df_m['60일 평균 종가 대비 당일 종가 비율'] = df_m['end_close']/df_m['60일 평균 종가']
        df_m['120일 평균 종가 대비 당일 종가 비율'] = df_m['end_close']/df_m['120일 평균 종가']

        df_m['5일평균거래량대비당일거래량'] = df_m['trading_volume']/df_m['5일 평균 거래량']
        df_m['10일평균거래량대비당일거래량'] = df_m['trading_volume']/df_m['10일 평균 거래량']
        df_m['20일평균거래량대비당일거래량'] = df_m['trading_volume']/df_m['20일 평균 거래량']
        df_m['60일평균거래량대비당일거래량'] = df_m['trading_volume']/df_m['60일 평균 거래량']
        df_m['120일평균거래량대비당일거래량'] = df_m['trading_volume']/df_m['120일 평균 거래량']

        df_m['코스피지수5일평균종가대비당일종가비율'] = df_m['kospi']/df_m['코스피 5일 평균 종가']
        df_m['코스피지수20일평균종가대비당일종가비율'] = df_m['kospi']/df_m['코스피 20일 평균 종가']
        df_m['코스피지수60일평균종가대비당일종가비율'] = df_m['kospi']/df_m['코스피 60일 평균 종가']
        df_m['코스피지수120일평균종가대비당일종가비율'] = df_m['kospi']/df_m['코스피 120일 평균 종가']
        
        target = ['X1일 뒤 X2일 평균 종가 변화량']
        f_col = ['start_open', 'high', 'low', 'end_close',
               'trading_volume', 'transaction_amount', 'end_rate_change',
               'institutional_total', 'other_corporations', 'individual',
               'foreigner_total', 'short_selling', 'short_buying', 'short_importance',
               '1yd_end_close', '1yd_trading', '5일 평균 종가', '10일 평균 종가', '20일 평균 종가',
               '60일 평균 종가', '120일 평균 종가', '5일 평균 거래량', '10일 평균 거래량', '20일 평균 거래량',
               '60일 평균 거래량', '120일 평균 거래량', 'kospi','inflation', 'treasury_bonds', 
                 'tightening','normality', 'powell', 'dispute', 'japan', 'volume', 'chairman',
                 'remarks', 'thought', 'effect', 'anxiety', 'buying', 
                 'volatility', 'early_stage', 'decline', 'learning_result',
               'america_top_500', 'gold', 'copper', 'k_gov3', 'usd_k', '코스피 5일 평균 종가',
               '코스피 20일 평균 종가', '코스피 60일 평균 종가', '코스피 120일 평균 종가', '전일종가대비 당일 시가비율',
               '당일종가 대비 당일 고가 비율', '당일종가 대비 당일 저가 비율', '당일 종가 대비 전일 종가 비율',
               '전일 거래량 대비 당일 거래량 비율', '5일 평균 종가 대비 당일 종가 비율', '10일 평균 종가 대비 당일 종가 비율',
               '20일 평균 종가 대비 당일 종가 비율', '60일 평균 종가 대비 당일 종가 비율',
               '120일 평균 종가 대비 당일 종가 비율', '5일평균거래량대비당일거래량', '10일평균거래량대비당일거래량',
               '20일평균거래량대비당일거래량', '60일평균거래량대비당일거래량', '120일평균거래량대비당일거래량',
               '코스피지수5일평균종가대비당일종가비율', '코스피지수20일평균종가대비당일종가비율', 
                 '코스피지수60일평균종가대비당일종가비율','코스피지수120일평균종가대비당일종가비율']
        
        train_x, test_x, train_y, test_y = train_test_split(df_m[f_col], df_m[target], test_size=0.15,shuffle=False)
        
        #xgbboost
        xgb = XGBClassifier(n_estimators=200)
        xgb.fit(train_x, train_y)
        y_pred = xgb.predict(test_x)
        train_pred = xgb.predict(train_x)
        
        total = total + f1_score(test_y, y_pred, average='macro')
        
        joblib.dump(xgb, 'model/'+name+'_model.pkl')
        print("AI 예측 정확도 - " + name + ":",f1_score(test_y, y_pred, average='macro'))
    
    print("종목 총합 정확도 : ",(total/stock_n))

finally:
    conn.close()

AI 예측 정확도 - 메리츠화재: 0.3802842109031034
AI 예측 정확도 - 유한양행: 0.6148252772546559
AI 예측 정확도 - 기아: 0.6028546166477202
AI 예측 정확도 - SK하이닉스: 0.36332422941865006
AI 예측 정확도 - 현대건설: 0.6561908679555738
AI 예측 정확도 - 삼성화재: 0.668545803971813
AI 예측 정확도 - 대한항공: 0.558972972972973
AI 예측 정확도 - LG: 0.43570601851851853
AI 예측 정확도 - 포스코케미칼: 0.3396681749622926
AI 예측 정확도 - 현대제철: 0.576325314030232
AI 예측 정확도 - 롯데지주: 0.538941365175548
AI 예측 정확도 - 현대차: 0.6568269425847029
AI 예측 정확도 - POSCO홀딩스: 0.5290183165260341
AI 예측 정확도 - DB손해보험: 0.6808055605061487
AI 예측 정확도 - 삼성전자: 0.5657334468809878
AI 예측 정확도 - 삼성전자우: 0.477268134976763
AI 예측 정확도 - NH투자증권: 0.37975987207632056
AI 예측 정확도 - 삼성SDI: 0.42816091954022983
AI 예측 정확도 - 미래에셋증권: 0.44024110218140067
AI 예측 정확도 - GS리테일: 0.5464782690296807
AI 예측 정확도 - 메리츠증권: 0.5849104599104599
AI 예측 정확도 - 호텔신라: 0.42844655344655336
AI 예측 정확도 - 삼성전기: 0.3570918031849501
AI 예측 정확도 - 한국조선해양: 0.6635453244895305
AI 예측 정확도 - 한화솔루션: 0.444464243288192
AI 예측 정확도 - 고려아연: 0.4254389387436598
AI 예측 정확도 - 삼성중공업: 0.

In [20]:
import datetime

#오늘 날짜를 표시 
dt_today = datetime.date.today() 

print(dt_today.strftime('%Y-%m-%d'))
#2022-05-10


2022-12-26


In [15]:
conn = pymysql.connect(host='localhost', user='root', password='a026195', db='stockai', charset='utf8', cursorclass=pymysql.cursors.DictCursor)
curs = conn.cursor()
stock_code_sql = "SELECT DISTINCT stock_code FROM StockData"
curs.execute(stock_code_sql)
stock_code = curs.fetchall()
stock_code_list = pd.DataFrame(stock_code)


#출력하고자 하는 주식 개수 입력, 최대 100개
stock_n = 100

try:
    # for문에 
    for i in range(0,stock_n):

        curs = conn.cursor()
        sql = "SELECT * FROM StockData WHERE (data_time BETWEEN '2022-06-01' AND '2022-12-08') AND (stock_code ='{}')".format(str(stock_code_list['stock_code'][i]))
        # 코드 앞에서 A 지우고 주식 이름 불러오기
        name = stock.get_market_ticker_name(str(stock_code_list['stock_code'][i][1:]))
        curs.execute(sql)
        data = curs.fetchall()
        
        df = pd.DataFrame(data)
        
        df = df[-120:]
        
        # 0.00등 Nan 값으로 인식되지 않는 결측치를 Nan으로 바꿈
        df["start_open"] = df["start_open"].apply(lambda x: np.nan if x<0.00001 else x)
        df["high"] = df["high"].apply(lambda x: np.nan if x<0.00001 else x)
        df["low"] = df["low"].apply(lambda x: np.nan if x<0.00001 else x)
        df["trading_volume"] = df["trading_volume"].apply(lambda x: np.nan if x<0.00001 else x)
        df["institutional_total"] = df["institutional_total"].apply(lambda x: np.nan if x<0.00001 else x)
        df["other_corporations"] = df["other_corporations"].apply(lambda x: np.nan if x<0.00001 else x)
        df["individual"] = df["individual"].apply(lambda x: np.nan if x<0.00001 else x)
        df["foreigner_total"] = df["foreigner_total"].apply(lambda x: np.nan if x<0.00001 else x)
        df["transaction_amount"] = df["transaction_amount"].apply(lambda x: np.nan if x<0.00001 else x)
        df["short_selling"] = df["short_selling"].apply(lambda x: np.nan if x<0.01 else x)
        df["short_buying"] = df["short_buying"].apply(lambda x: np.nan if x<0.01 else x)
          
        df = df.fillna(method='bfill')
        df = df.fillna(method='ffill')
    
        df['1yd_end_close'] = df['end_close'].shift(1)
        df['1yd_trading'] = df['trading_volume'].shift(1)
        df['5일 평균 종가'] = df['end_close'].rolling(5).mean()
        df['10일 평균 종가'] = df['end_close'].rolling(10).mean()
        df['20일 평균 종가'] = df['end_close'].rolling(20).mean()
        df['60일 평균 종가'] = df['end_close'].rolling(60).mean()
        df['120일 평균 종가'] = df['end_close'].rolling(120).mean()
    
        df['5일 평균 거래량'] = df['trading_volume'].rolling(5).mean()
        df['10일 평균 거래량'] = df['trading_volume'].rolling(10).mean()
        df['20일 평균 거래량'] = df['trading_volume'].rolling(20).mean()
        df['60일 평균 거래량'] = df['trading_volume'].rolling(60).mean()
        df['120일 평균 거래량'] = df['trading_volume'].rolling(120).mean()
    
        pd.set_option('mode.chained_assignment',  None)

        df.dropna(inplace=True)
        
        sql2 = "SELECT * FROM MacroeconomicIndicators WHERE (date_time BETWEEN '2022-06-01' AND '2022-12-08')"
        curs.execute(sql2)
        data2 = curs.fetchall()

        df2 = pd.DataFrame(data2)
        
        df2 = df2[-120:]

        df2['코스피 5일 평균 종가'] = df2['kospi'].rolling(5).mean()
        df2['코스피 20일 평균 종가'] = df2['kospi'].rolling(20).mean()
        df2['코스피 60일 평균 종가'] = df2['kospi'].rolling(60).mean()
        df2['코스피 120일 평균 종가'] = df2['kospi'].rolling(120).mean()
    
        df2.dropna(inplace=True)
        
        df_m = pd.merge(df, df2, left_on='data_time',right_on='date_time',how='inner')
        
        #merge하면서 생기는 id 행들 제거
        df_m.drop(columns=['id_x','id_y'],inplace=True)
                                            
        # 컬럼 제작
        df_m['전일종가대비 당일 시가비율'] = df_m['1yd_end_close']/df_m['start_open']
        df_m['당일종가 대비 당일 고가 비율'] = df_m['high']/df_m['end_close']
        df_m['당일종가 대비 당일 저가 비율'] = df_m['low']/df_m['end_close']
        df_m['당일 종가 대비 전일 종가 비율'] = df_m['end_close']/df_m['1yd_end_close']
        df_m['전일 거래량 대비 당일 거래량 비율'] = df_m['trading_volume']/df_m['1yd_trading']
        df_m['5일 평균 종가 대비 당일 종가 비율'] = df_m['end_close']/df_m['5일 평균 종가']
        df_m['10일 평균 종가 대비 당일 종가 비율'] = df_m['end_close']/df_m['10일 평균 종가']
        df_m['20일 평균 종가 대비 당일 종가 비율'] = df_m['end_close']/df_m['20일 평균 종가']
        df_m['60일 평균 종가 대비 당일 종가 비율'] = df_m['end_close']/df_m['60일 평균 종가']
        df_m['120일 평균 종가 대비 당일 종가 비율'] = df_m['end_close']/df_m['120일 평균 종가']

        df_m['5일평균거래량대비당일거래량'] = df_m['trading_volume']/df_m['5일 평균 거래량']
        df_m['10일평균거래량대비당일거래량'] = df_m['trading_volume']/df_m['10일 평균 거래량']
        df_m['20일평균거래량대비당일거래량'] = df_m['trading_volume']/df_m['20일 평균 거래량']
        df_m['60일평균거래량대비당일거래량'] = df_m['trading_volume']/df_m['60일 평균 거래량']
        df_m['120일평균거래량대비당일거래량'] = df_m['trading_volume']/df_m['120일 평균 거래량']

        df_m['코스피지수5일평균종가대비당일종가비율'] = df_m['kospi']/df_m['코스피 5일 평균 종가']
        df_m['코스피지수20일평균종가대비당일종가비율'] = df_m['kospi']/df_m['코스피 20일 평균 종가']
        df_m['코스피지수60일평균종가대비당일종가비율'] = df_m['kospi']/df_m['코스피 60일 평균 종가']
        df_m['코스피지수120일평균종가대비당일종가비율'] = df_m['kospi']/df_m['코스피 120일 평균 종가']
        
        f_col = ['start_open', 'high', 'low', 'end_close',
               'trading_volume', 'transaction_amount', 'end_rate_change',
               'institutional_total', 'other_corporations', 'individual',
               'foreigner_total', 'short_selling', 'short_buying', 'short_importance',
               '1yd_end_close', '1yd_trading', '5일 평균 종가', '10일 평균 종가', '20일 평균 종가',
               '60일 평균 종가', '120일 평균 종가', '5일 평균 거래량', '10일 평균 거래량', '20일 평균 거래량',
               '60일 평균 거래량', '120일 평균 거래량', 'kospi', 
                 'inflation', 'treasury_bonds', 'tightening','normality', 
                 'powell', 'dispute', 'japan', 'volume', 'chairman',
                 'remarks', 'thought', 'effect', 'anxiety', 'buying', 
                 'volatility', 'early_stage', 'decline', 'learning_result',
               'america_top_500', 'gold', 'copper', 'k_gov3', 'usd_k', '코스피 5일 평균 종가',
               '코스피 20일 평균 종가', '코스피 60일 평균 종가', '코스피 120일 평균 종가', '전일종가대비 당일 시가비율',
               '당일종가 대비 당일 고가 비율', '당일종가 대비 당일 저가 비율', '당일 종가 대비 전일 종가 비율',
               '전일 거래량 대비 당일 거래량 비율', '5일 평균 종가 대비 당일 종가 비율', '10일 평균 종가 대비 당일 종가 비율',
               '20일 평균 종가 대비 당일 종가 비율', '60일 평균 종가 대비 당일 종가 비율',
               '120일 평균 종가 대비 당일 종가 비율', '5일평균거래량대비당일거래량', '10일평균거래량대비당일거래량',
               '20일평균거래량대비당일거래량', '60일평균거래량대비당일거래량', '120일평균거래량대비당일거래량',
               '코스피지수5일평균종가대비당일종가비율', '코스피지수20일평균종가대비당일종가비율', 
                 '코스피지수60일평균종가대비당일종가비율','코스피지수120일평균종가대비당일종가비율']
        
        test_x = df_m[f_col]
        
        model = joblib.load('model/'+name+'_model.pkl')

        y_pred = model.predict(test_x)

        print()
        print(name + "의 " + str(df_m['data_time'][0])+" 예측 결과 = ",y_pred)
finally:
    conn.close()


메리츠화재의 2022-12-08 예측 결과 =  [0]

유한양행의 2022-12-08 예측 결과 =  [0]

기아의 2022-12-08 예측 결과 =  [0]

SK하이닉스의 2022-12-08 예측 결과 =  [1]

현대건설의 2022-12-08 예측 결과 =  [0]

삼성화재의 2022-12-08 예측 결과 =  [0]

대한항공의 2022-12-08 예측 결과 =  [0]

LG의 2022-12-08 예측 결과 =  [0]

포스코케미칼의 2022-12-08 예측 결과 =  [0]

현대제철의 2022-12-08 예측 결과 =  [0]

롯데지주의 2022-12-08 예측 결과 =  [0]

현대차의 2022-12-08 예측 결과 =  [0]

POSCO홀딩스의 2022-12-08 예측 결과 =  [0]

DB손해보험의 2022-12-08 예측 결과 =  [0]

삼성전자의 2022-12-08 예측 결과 =  [0]

삼성전자우의 2022-12-08 예측 결과 =  [0]

NH투자증권의 2022-12-08 예측 결과 =  [1]

삼성SDI의 2022-12-08 예측 결과 =  [0]

미래에셋증권의 2022-12-08 예측 결과 =  [1]

GS리테일의 2022-12-08 예측 결과 =  [0]

메리츠증권의 2022-12-08 예측 결과 =  [0]

호텔신라의 2022-12-08 예측 결과 =  [1]

삼성전기의 2022-12-08 예측 결과 =  [1]

한국조선해양의 2022-12-08 예측 결과 =  [1]

한화솔루션의 2022-12-08 예측 결과 =  [0]

고려아연의 2022-12-08 예측 결과 =  [0]

삼성중공업의 2022-12-08 예측 결과 =  [1]

현대미포조선의 2022-12-08 예측 결과 =  [1]

S-Oil의 2022-12-08 예측 결과 =  [1]

LG이노텍의 2022-12-08 예측 결과 =  [0]

롯데케미칼의 2022-12-08 예측 결과 =  [0]

HMM의 2022-12-08

In [17]:
df_m

,stock_code,data_time,start_open,high,low,end_close,trading_volume,transaction_amount,end_rate_change,institutional_total,...,120일 평균 종가 대비 당일 종가 비율,5일평균거래량대비당일거래량,10일평균거래량대비당일거래량,20일평균거래량대비당일거래량,60일평균거래량대비당일거래량,120일평균거래량대비당일거래량,코스피지수5일평균종가대비당일종가비율,코스피지수20일평균종가대비당일종가비율,코스피지수60일평균종가대비당일종가비율,코스피지수120일평균종가대비당일종가비율
0,A402340,2022-12-08,35200.0,35750.0,35050.0,35600.0,541102.0,1.921480e+10,1.14,1.965450e+09,...,0.897734,1.40218,1.224077,1.430081,1.468033,1.414641,0.98544,0.970493,1.014019,0.996376
